In this homework we'll put what we learned about Spark in practice.

For this homework we will be using the Yellow 2024-10 data from the official website:
```
    wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
```
Note: I put this file in the `'data/raw/yellow/2024/10'` folder

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

## Question 1: Check version after installing Spark and PySpark

In [2]:
spark.version

'3.3.2'

## Question 2: Yellow October 2024

Read the October 2024 Yellow into a Spark Dataframe.

Repartition the Dataframe to 4 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

In [79]:
# read the file into a spark dataframe
df = spark.read.parquet('data/raw/yellow/2024/10')

In [6]:
# repartition into 4 partitions and save
df.repartition(4).write.parquet('data/pq/yellow/2024/10')

In [20]:
# get average filesize
import os
import glob
total = 0
for file_path in glob.glob(os.path.join('data/pq/yellow/2024/10', '*.parquet')):
    total += os.stat(file_path).st_size
print(f"Average: {(total/4)} bytes") 

Average: 25406383.75 bytes


which is about 25 MB.

## Question 3: Count records

How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

In [71]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [76]:
df.filter(F.date_trunc('day', df.tpep_pickup_datetime) == '2024-10-15') \
    .count()

116396

## Question 4: Longest trip

What is the length of the longest trip in the dataset in hours?

In [48]:
result = df.withColumn('trip_duration', df.tpep_dropoff_datetime - df.tpep_pickup_datetime) \
    .sort('trip_duration', ascending=False) \
    .select('trip_duration') \
    .take(1)

In [57]:
print(f"Num hours for longest trip: {(result[0].trip_duration.total_seconds() / 3600):.2f}")

Num hours for longest trip: 162.62


## Question 5: User Interface

Spark’s User Interface which shows the application's dashboard runs on which local port?

In [70]:
ui_url = spark.sparkContext.uiWebUrl
print("Port Number: ", ui_url.split(':')[-1])

Port Number:  4040


## Question 6: Least frequent pickup location zone

Load the zone lookup data into a temp view in Spark:
```
wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
```
Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

In [78]:
# Load zone lookup into temp view
df_zones = spark.read \
    .option("header", "true") \
    .csv('data/raw/taxi_zone_lookup.csv')
df_zones.createOrReplaceTempView('zones')

In [81]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [80]:
# Create temp view from yellow trip data
df.createOrReplaceTempView('yellow_trips')

In [98]:
# Write query to find least frequent pickup zone
df_result = spark.sql("""
SELECT
    z.Zone,
    count(*) as num_trips
FROM 
    yellow_trips y
    INNER JOIN zones z
    ON y.PULocationID = z.LocationID
GROUP BY
    z.Zone
ORDER BY
    num_trips ASC
LIMIT 
    1
""")

In [99]:
df_result.take(1)

[Row(Zone="Governor's Island/Ellis Island/Liberty Island", num_trips=1)]